# this script allows to print all the information for a specific SNP

In [1]:
import pandas as pd
import numpy as np
import hicstraw 
from multiprocessing import Pool
from functools import partial
import glob
import os
import plotly.express as px
import math
import matplotlib.pyplot as plt
from matplotlib import colors
from pandarallel import pandarallel
import cooler
import cooltools
import pybedtools as pbed
pandarallel.initialize()
from scipy import stats, special
from statsmodels.stats import multitest
import statsmodels.api as sm
import statsmodels.formula.api as smf
import plotly.io as pio
import seaborn as sns

from functools import reduce

os.makedirs("/mnt/iusers01/jw01/mdefscs4/scratch/temp_pybedtools/", exist_ok = True)
pbed.helpers.set_tempdir("/mnt/iusers01/jw01/mdefscs4/scratch/temp_pybedtools/")
bed_genome_file = "/mnt/iusers01/jw01/mdefscs4/hg38.genome"

plt.rcParams['svg.fonttype'] = 'none'

base_dir = "/mnt/jw01-aruk-home01/projects/psa_functional_genomics/PsA_cleaned_analysis"

INFO: Pandarallel will run on 28 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


In [2]:
gtf_annotation_df = pd.read_pickle(f"{base_dir}/gencode_gtf.pickle")
gtf_transcripts = gtf_annotation_df[(gtf_annotation_df["feature"] == "transcript") & (gtf_annotation_df["transcript_type"] == "protein_coding")].dropna(axis=1, how='all')
gtf_transcripts["gene_id"] = gtf_transcripts["gene_id"].str.split(".").str[0]
gtf_transcripts["transcript_id"] = gtf_transcripts["transcript_id"].str.split(".").str[0]
gtf_transcripts["TSS_start"] = gtf_transcripts.apply(lambda x: int(x["start"]) if x["strand"] == "+" else int(x["end"]) ,axis = 1)
gene_mapper = dict(zip(gtf_transcripts['gene_id'], gtf_transcripts['gene_name']))

In [3]:
RNA_nominal_CD4 = pd.read_csv(f"{base_dir}/QTL_analysis/RNA/output_final/RNA_nominal_CD4_merged.txt", sep = " ")
RNA_nominal_CD8 = pd.read_csv(f"{base_dir}/QTL_analysis/RNA/output_final/RNA_nominal_CD8_merged.txt", sep = " ")
ATAC_nominal_CD8 = pd.read_csv(f"{base_dir}/QTL_analysis/ATAC/output_final/ATAC_nominal_CD8_merged.txt", sep = " ")
ATAC_nominal_CD4 = pd.read_csv(f"{base_dir}/QTL_analysis/ATAC/output_final/ATAC_nominal_CD4_merged.txt", sep = " ")
ins_nominal_CD8 = pd.read_csv(f"{base_dir}/QTL_analysis/HiC/output_final/ins_nominal_CD8_merged.txt", sep = " ")
ins_nominal_CD4 = pd.read_csv(f"{base_dir}/QTL_analysis/HiC/output_final/ins_nominal_CD4_merged.txt", sep = " ")
loop_nominal_CD8 = pd.read_csv(f"{base_dir}/QTL_analysis/HiC/output_final/loop_nominal_CD8_merged.txt", sep = " ")
loop_nominal_CD4 = pd.read_csv(f"{base_dir}/QTL_analysis/HiC/output_final/loop_nominal_CD4_merged.txt", sep = " ")

output_dataframe_CD8 = pd.read_csv(f"{base_dir}/HiC_allelic_imbalance/output_dataframe_CD8.csv", index_col=0)
output_dataframe_CD4 = pd.read_csv(f"{base_dir}/HiC_allelic_imbalance/output_dataframe_CD4.csv", index_col=0)

o = f"{base_dir}/ATAC_allelic_imbalance/combined_p_vals_files/"
all_SNPs_all = pd.read_pickle(o + "all_SNPs_all.pkl").drop(columns = ["eQTLgen_gene", "snp","hsc_genes","tcell_genes","all_genes"])

In [4]:
RNA_nominal_CD4["gene_name"] = RNA_nominal_CD4['phe_id'].map(gene_mapper)
RNA_nominal_CD8["gene_name"] = RNA_nominal_CD8['phe_id'].map(gene_mapper)

In [5]:
def get_all_tables(vars):
    if not type(vars) == list:
        vars = [vars]
    print("eQTL CD4")
    display(RNA_nominal_CD4[RNA_nominal_CD4["var_id"].isin(vars)])

    print("eQTL CD8")
    display(RNA_nominal_CD8[RNA_nominal_CD8["var_id"].isin(vars)])

    print("caQTL CD4")
    display(ATAC_nominal_CD4[ATAC_nominal_CD4["var_id"].isin(vars)])

    print("caQTL CD8")
    display(ATAC_nominal_CD8[ATAC_nominal_CD8["var_id"].isin(vars)])

    print("loopQTL CD4")
    display(loop_nominal_CD4[loop_nominal_CD4["var_id"].isin(vars)])

    print("loopQTL CD8")
    display(loop_nominal_CD8[loop_nominal_CD8["var_id"].isin(vars)])

    print("insQTL CD4")
    display(ins_nominal_CD4[ins_nominal_CD4["var_id"].isin(vars)])

    print("insQTL CD8")
    display(ins_nominal_CD8[ins_nominal_CD8["var_id"].isin(vars)])


    print("allelic imbalance")
    display(all_SNPs_all[all_SNPs_all["ID"].isin(vars)])

    print("loops with allelic imbalance CD4")
    display(output_dataframe_CD4[output_dataframe_CD4["rsID"].isin(vars)])

    print("loops with allelic imbalance CD8")
    display(output_dataframe_CD8[output_dataframe_CD8["rsID"].isin(vars)])

In [6]:
get_all_tables("rs13401811")

eQTL CD4


,phe_id,phe_chr,phe_from,phe_to,phe_strd,n_var_in_cis,dist_phe_var,var_id,var_chr,var_from,var_to,nom_pval,r_squared,slope,best_hit,gene_name


eQTL CD8


,phe_id,phe_chr,phe_from,phe_to,phe_strd,n_var_in_cis,dist_phe_var,var_id,var_chr,var_from,var_to,nom_pval,r_squared,slope,best_hit,gene_name


caQTL CD4


...

,CHROM,POS,ID,REF,ALT,combined_p_val_greater,combined_p_val_less,tot_REF,tot_ALT,ratio,n_pat,corrected_p_val_greater,corrected_p_val_less,TF_remap,TF_JASPAR,eQTLgen_symbol,eQTLgen_pval,CD4_lowest_allele_specific,CD8_lowest_allele_specific,ATAC_hic_corr_score
25963,chr2,110858527,rs13401811,G,A,1.594457e-98,1.0,3724.0,2207.0,0.592642,29.0,1.856879e-96,1.0,"{GABPA, JUNB, SPI1, SKIL, FOXM1, ZFP36, BHLHE4...","{ZEB1, SNAI1, NR4A2, GATA4, JUNB, TCF3, Ptf1A,...",NaN,NaN,NaN,NaN,0.1


loops with allelic imbalance CD4


,chrA,startA,endA,chrB,startB,endB,loopID,loopScore,rsID,rsCoord,combined_Pval_greater,corrected_Pval_greater,combined_Pval_less,corrected_Pval_less,tot_REF,tot_ALT,n_pat


loops with allelic imbalance CD8


,chrA,startA,endA,chrB,startB,endB,loopID,loopScore,rsID,rsCoord,combined_Pval_greater,corrected_Pval_greater,combined_Pval_less,corrected_Pval_less,tot_REF,tot_ALT,n_pat
